# Image Classification: dogs vs cats

Download Dataset [here](https://www.kaggle.com/c/dogs-vs-cats/data)

**Organize the dataset**
 
Download datset and delete test.zip and csv files. We don't need it in this example.
Make sure all images are inside of one fodler like:
 
         /data/dogs-vs-cats/train/
         
The recommendation here is try to run the code using a GPU to make it fast. 

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

# Data Preparation

The images included in the data/cats-and-dogs directory are a random subset of the full cat and dog data set from the Kaggle competion.

In [ ]:
# Organize data into train, valid, test dirs
os.chdir('data/dogs-vs-cats')
if os.path.isdir('train/dog') is False
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
    # Train dataset
    for c in random.sample(glob.glob('cat*'),500):
        shutil.move(c, 'train/cat')
    for c in random.sample(glob.glob('dog*'),500):
        shutil.move(c, 'train/dog')
    
    # Validation set
    for c in random.sample(glob.glob('cat*'),100):
        shutil.move(c, 'valid/cat')
    for c in random.sample(glob.glob('dog*'),100):
        shutil.move(c, 'valid/dog')
    
    # Test set
    for c in random.sample(glob.glob('cat*'),50):
        shutil.move(c, 'test/cat')
    for c in random.sample(glob.glob('dog*'),50):
        shutil.move(c, 'test/dog')

In [ ]:
train_path = 'data/dogs-vs-cats/train'
valid_path = 'data/dogs-vs-cats/valid'
test_path = 'data/dogs-vs-cats/test'

# Set up the data format suitable to Keras

Here we are processing the images in the same format the VGG16 model process. 

**Suffle is False in Test Batches**

It is because whenever we use our test batches later for inference to get our model to predict on images of cats and dogs after training and validation has been completed we are going to want to look at out prediction results in a confusion matrix and in order to do that we need to be able to access the unsheffuled labels for our test set

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Verification that we have the correct amount of data

In [ ]:
assert train_bataches.n == 1000
assert valid_bataches.n == 200
assert test_bataches.n == 100
assert train_batches.num_classes = valid_batches.num_classes = test_batches.num_classes = 2

We now call next(train_batches) to generate a batch of images and labels from the training set. 

In [ ]:
# we will get 10 images and 10 correspnd lables
imgs, labels = next(train_batches)

**Plot Images**

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

# Build CNN using Tensorflow and Keras API

**[Zero padding](https://deeplizard.com/learn/video/qSTv_m-KFk0)**

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(), # flatt alll of this into 1-D tensor 
    Dense(units=2, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics['accuracy'])

Here we don't specify the labels because when the data is stored as generator as a generator, the generator itself actually contains the corresponding lables, so we don't need to specify them separately whenever we call fit.

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

**Observation**

The training accuracy is much higher than validation accuracy thats means the model not generalized well and we have overfitting in training. 

# Predict

In [ ]:
test_imgs, test_lables = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

**Interpreting predictions output**

- [1., 0.] : it means the model predict the class in ZERO index that has the value 1. -> CAT
- [0., 1.] : it means the model predict the class in ONE index that has the value 1. -> DOG


## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmx(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Extract the labels to use in the right sequence for our confusion matrix
test_batches.class_indices

In [ ]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion matrix')

# Fine-Tunning to increase the performance of model

The model that we uses here is VGG16 that won's the ImageNet Compettion at 2014.

The ImageNet library is made up of thousands of images that belong to one thousand of different classes.

Using Keras we will import this VGG16 model and then fine tune it to not classify on one of the 1000 categories for which it was originally trained but instead only on two categories, cat and dog. 

Note, however that cats and dogs were included in the original image net library for wich VGG16 was trained on and becasue of this we sont have to do much tunning to change the model from classyfing from 1000 classes to  just the to cat and dog classes. So the overal gine-tunning the we'll do will be very minimal. 

In [ ]:
# Download model - Internete conenction nedded
vgg16_model = tf.keras.application.vgg16.VGG16()

In [ ]:
# to see the mdeol architecture
vgg16_model.summary()

We can see in the alst layer (output layer) the model was built to rpedict 1000 classe.s Our work here is modify this output layer to model predict only two output classes correspondig cat and dogs.

In [ ]:
# to make sure we import the model in correct way
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shapr(.as_list()) for v in model.non_trainable_weights)])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [ ]:
params = count(vgg16_model)
assert params['non_trainable_params'] == 0
assert aprams['trainable_params'] == 138357544

In [ ]:
# the type of vgg16_model is Model type
type(vgg16_model)

We need to convert Model type into Sequential type.

Next we are loop all layer, except the last one (prediction layer), and add each layer in our Sequential model.

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

**Why FREZZ Weights and Biases?**

Now, we need to inform our model that the layers that are in the model are not trainable layers. In other words we are saying, don't touch on weights and biases values thar are in layers because they're not going to be retrained whenever we go through the training process for cats and dogs. 

Becasue VGG16 has already learned features of cats and dogs in its original training we don't wanted to have to go through more training again since it's already learned those features. This is the reason to freez the weights. 

In [ ]:
for layer in model.layers:
    layer.trainable = False

**Add our output layer** to this model. Remember we've removed the previous output layer.

In [ ]:
model.add(Dense(units=2, activation='softmax'))

In [ ]:
# to confirm the classes of the otput layer
model.summary()

In [ ]:
# to make sure that our model was well build
params = count_params(model)
assert params['non_trainable_params'] == 134260544
assert params['trainable_params'] == 8194

# Train the fine-tuned VGG16 model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# fit model to our data
model.fit(x=train_batches, validation=valid_batches, epochs=5, verbose=2)

**Duscussion Results**

We get more accurated results with this new model. It is not totally surprising because the VGG16 had already been trained on images of cats and dogs from ImageNet library. So, the model had already those features. 

The improvement was gained when we told the VGG16 model to classify tje images only in two categories, cat or dog. 

In [ ]:
assert model.history.history.get('accuracy')[-1] > 0.95

# Predict using fine-tuned VGG16 model

In [ ]:
predictions = model.predict(x=test_batchs, verbose=0)

In [ ]:
test_batches.classes

In [ ]:
cm= confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions,axis=-1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion matrix')